In [3]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP

In [279]:
demand = CSV.File("demand.csv";header=false)|> Tables.matrix;
coordinate = CSV.File("coordinate.csv";header=false)|> Tables.matrix;
time_window = CSV.File("time_window.csv";header=false)|> Tables.matrix;
V_c = CSV.File("V_c.csv";header=false)|> Tables.matrix;

### Define Matrix

Under 4 customers:
- number_of_customers = 4,    
- number_of_locations = 15 (1 depot + 14 customer locations)
- time_horizon = 720
- vehicle_capacity = 750

There are 4 customers. Deliveries can be made at 15 locations (some of them may be physically the same place, but at different times). 
The time horizon is 720 minutes, and each vehicle can carry a load of up to 750 demand units.

Matrix:
* `Vc`: set of locations travelled by customer C & depot location (at index 1).    *(17 element vector)*
* `Kc`: the number of locations travelled by customer C.    *(17 element vector)*
* `aic` & `bic`: starting & ending time of time window for customer C at locaition i respectively.  *(17✖6 Matrix)*
* `t`: the distance between location i and j, which we define as the time travelling between location i and j  *(63✖63 Matrix)*

#### `Vc`

In [283]:
Vc = Array{Vector{Int}}(undef,size(V_c)[1]);
for i in 1:size(V_c)[1]
    element = V_c[i]
    if element isa String31
        Vc[i] = [parse(Int, s)+1 for s in split(element)]
    end
end
# Vc  

#### `Kc`

In [284]:
Kc = Array{Int64}(undef,size(Vc)[1])
for ind in 1:size(Vc)[1]
    Kc[ind] = size(Vc[ind])[1]
end
# Kc

In [312]:
Vc, Kc

([[1], [2], [3, 4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14], [15, 16, 17, 18], [19, 20, 21, 22, 23], [24], [25], [26, 27, 28, 29, 30, 31], [32, 33, 34], [35, 36, 37, 38, 39], [40, 41, 42, 43, 44], [45], [46, 47, 48, 49], [50], [51]], [1, 1, 5, 4, 3, 4, 5, 1, 1, 6, 3, 5, 5, 1, 4, 1, 1])

#### `aic` & `bic` 

In [287]:
n,p = size(Kc)[1], maximum(Kc)

(17, 6)

In [288]:
timewindow = Array{Union{Missing, Vector{Int}}}(undef, n, p);
for i in 1:n
    for j in 1:p
        element = time_window[i,j]
        if element isa String7
            timewindow[i,j] = [parse(Int, s) for s in split(element)]
        else
            timewindow[i,j] = missing
        end
    end
end
# timewindow

In [289]:
aic = Array{Union{Missing, Int}}(undef, n, p);
bic = Array{Union{Missing, Int}}(undef, n, p);

In [290]:
# find all the a_i^c and b_i^c
for row_ind in 1:n
    for col_ind_notnull in 1:Kc[row_ind]
        aic[row_ind,col_ind_notnull] = timewindow[row_ind,col_ind_notnull][1]
        bic[row_ind,col_ind_notnull] = timewindow[row_ind,col_ind_notnull][2]
    end
end

In [291]:
aic

17×6 Matrix{Union{Missing, Int64}}:
 0     missing     missing     missing     missing     missing
 0     missing     missing     missing     missing     missing
 0  212         354         531         637            missing
 0  140         297         338            missing     missing
 0  332         664            missing     missing     missing
 0  256         475         621            missing     missing
 0  133         391         617         672            missing
 0     missing     missing     missing     missing     missing
 0     missing     missing     missing     missing     missing
 0   50         145         472         603         650
 0  312         592            missing     missing     missing
 0   83         341         506         652            missing
 0  178         320         417         633            missing
 0     missing     missing     missing     missing     missing
 0  263         497         666            missing     missing
 0     missing     missing

In [292]:
bic

17×6 Matrix{Union{Missing, Int64}}:
 720     missing     missing     missing     missing     missing
 720     missing     missing     missing     missing     missing
  65  213         370         542         720            missing
  87  236         325         720            missing     missing
   2  334         720            missing     missing     missing
  20  310         480         720            missing     missing
  27  150         440         670         720            missing
 720     missing     missing     missing     missing     missing
 720     missing     missing     missing     missing     missing
  35   78         289         479         625         720
  69  349         720            missing     missing     missing
  70  166         342         549         720            missing
 116  257         401         616         720            missing
 720     missing     missing     missing     missing     missing
  64  279         612         720            missing     miss

In [293]:
timewindow

17×6 Matrix{Union{Missing, Vector{Int64}}}:
 [0, 720]  missing     missing     missing     missing     missing
 [0, 720]  missing     missing     missing     missing     missing
 [0, 65]   [212, 213]  [354, 370]  [531, 542]  [637, 720]  missing
 [0, 87]   [140, 236]  [297, 325]  [338, 720]  missing     missing
 [0, 2]    [332, 334]  [664, 720]  missing     missing     missing
 [0, 20]   [256, 310]  [475, 480]  [621, 720]  missing     missing
 [0, 27]   [133, 150]  [391, 440]  [617, 670]  [672, 720]  missing
 [0, 720]  missing     missing     missing     missing     missing
 [0, 720]  missing     missing     missing     missing     missing
 [0, 35]   [50, 78]    [145, 289]  [472, 479]  [603, 625]  [650, 720]
 [0, 69]   [312, 349]  [592, 720]  missing     missing     missing
 [0, 70]   [83, 166]   [341, 342]  [506, 549]  [652, 720]  missing
 [0, 116]  [178, 257]  [320, 401]  [417, 616]  [633, 720]  missing
 [0, 720]  missing     missing     missing     missing     missing
 [0, 64]   [263

#### `t`

In [294]:
l = size(coordinate)[1]
t = Array{Float64}(undef,l,l);
for i in 1:l
    for j in 1:l
        t[i,j]=sqrt((coordinate[i,1]-coordinate[j,1])^2+(coordinate[i,2]-coordinate[j,2])^2);
    end
end

In [295]:
t = 0.5 .* t

51×51 Matrix{Float64}:
  0.0       4.81138  19.6966    56.2677  …   63.264    65.4246   0.0
  4.81138   0.0      20.1367    55.8477      58.5466   61.6713   4.81138
 19.6966   20.1367    0.0       36.7236      69.2032   78.5627  19.6966
 56.2677   55.8477   36.7236     0.0         87.162   105.456   56.2677
 30.2325   27.3202   19.4941    35.2654      54.7709   70.3375  30.2325
 37.792    40.4688   23.6833    35.4347  …   92.8084  101.492   37.792
 19.6966   20.1367    0.0       36.7236      69.2032   78.5627  19.6966
 11.6738   10.3424   30.4027    66.1406      56.6931   55.042   11.6738
  3.58395   3.54447  17.1583    53.3846      61.446    65.1849   3.58395
 14.1743   11.8581   31.941     67.0185      53.4548   51.8541  14.1743
 11.6738   10.3424   30.4027    66.1406  …   56.6931   55.042   11.6738
  1.53036   6.29689  19.3768    56.0621      64.7885   66.8514   1.53036
 83.7047   86.1707   67.6149    51.3602     133.656   146.135   83.7047
  ⋮                                      ⋱

In [319]:
using Statistics

In [316]:
maximum(sum(t, dims=1))

5309.874854826538

In [323]:
(5309.874854826538 - 2128)/5309.874854826538

0.5992372592235947

In [324]:
(2899.2130196331777 -2128)/2899.2130196331777

0.2660077111997639

In [320]:
median(sum(t, dims=1))

2706.6306491456535

In [321]:
mean(sum(t, dims=1))

2899.2130196331777

In [296]:
maximum(t)

186.36777634405115

## Model Implementation

In [299]:
V = [1:size(coordinate)[1];];    # 1+14 locations traveled by customer
C = [1:length(demand);];         # 2+4 customers, 1 is the dummy

Z = [1:5;];     # number of vehicles
T = 720;        # max time
Q = 750000;     # max quantity per car

In [300]:
model = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [301]:
@variable(model, x[i in V, j in V, z in Z], Bin); # x_ijz
@variable(model, tau[c in C, z in Z] >= 0); # tau_cz
@variable(model, y[z in Z, c in C] >= 0); # y_zc

Each customer c $\in$ C visited location $V_c$ ($\subseteq V$). The data should look sth like:

- customer 1: [1] # dummy
- customer 2: [2]
- customer 3: [3,4,5,6,7]
- customer 4: [8,9,10,11]

Note, we assume all customers visited different places.

#### (1)

In [302]:
@constraint(model, [i in V, z in Z], sum(x[i,j,z] for j in V if j != i) == sum(x[j,i,z] for j in V if j != i));

In [16]:
# for i in V
#     @constraint(model, sum(x[i,j] for j in filter(x->x!=i,V)) == sum(x[j,i] for j in filter(x->x!=i,V)));
# end

#### (2)

In [303]:
@constraint(model, [c in C[2:end]], sum(sum(x[i,j,z] for i in Vc[c], j in V if j != i) for z in Z) == 1);

In [17]:
# for c in C[2:end]
#     Vc_c = hcat([location for location in Vc])[c] # all the nodes traveled by customer c
#     sum_xij_c = 0
#     for i in Vc_c
#         for j in V if j != i
#             sum_xij_c += x[i,j]
#         end
#     end
#     @constraint(model, sum_xij_c == 1);
# end

#### (3)

In [90]:
# for c in C[2:end]
#     @constraint(model, [i in 1:Kc[c]], sum(aic[c,i] * sum(x[i,j] for j in V if j != i)) <= tau[c]);
#     @constraint(model, [i in 1:Kc[c]], sum(bic[c,i] * sum(x[i,j] for j in V if j != i)) >= tau[c]);
# end

In [304]:
for c in 2:length(C),z in Z
        sum_aic = 0
        sum_bic = 0
        for i in 1:Kc[c]           # Kc[c] is the number of locations visited by customer c
            # V_no_i = V[1:length(V) .!= i]
            sum_aic += aic[c,i] * sum(x[i,j,z] for j in V if j != i)
            sum_bic += bic[c,i] * sum(x[i,j,z] for j in V if j != i)
        end
        @constraint(model, sum_aic <= tau[c,z])
        @constraint(model, sum_bic >= tau[c,z])
end

#### (4)

In [127]:
# for c in C
#     @constraint(model, [c2 in C[1:length(C) .!= c]],  tau[c] + sum(t[i,j] * x[i,j] for i in Vc[c], j in Vc[c2]) <= tau[c2] + T * ( 1- sum(x[i,j] for i in Vc[c], j in Vc[c2])));
# end

In [305]:
for c in C, z in Z
    C_no_c = C[1:length(C) .!= c]
    for c2 in C_no_c
        @constraint(model, tau[c,z] + sum(t[i,j]*x[i,j,z] for i in Vc[c], j in Vc[c2]) <= tau[c2,z] + T * ( 1- sum(x[i,j,z] for i in Vc[c], j in Vc[c2])));
    end
end

#### (5)

In [112]:
# for c in C
#      @constraint(model, [z in Z, c2 in C[1:length(C) .!= c]], y[z,c] + Q * ( 1- sum(x[i,j,z] for i in Vc[c], j in Vc[c2]) )>= demand[c2] + y[z,c2])
# end

In [23]:
for c in C
    C_no_c = C[1:length(C) .!= c]
    for c2 in C_no_c
        for z in 1:Z
            @constraint(model, y[z,c] + Q * ( 1- sum(x[i,j,z] for i in Vc[c], j in Vc[c2]) )>= demand[c2] + y[z,c2])
        end
    end
end

#### (6)

In [306]:
@constraint(model, sum(x[1,i,z] for i in V, z in Z) <= length(Z));

#### (7)

In [259]:
# @constraint(model, [c in C, z in 1:Z], y[z,c] <= Q-demand[c]);

In [22]:
# for c in C
#     for z in 1:Z
#         @constraint(model, y[z,c]>=0)
#         @constraint(model, y[z,c]<=Q-demand[c])
#     end
# end

In [307]:
@constraint(model, [z in Z], sum(x[1,j,z] for j in V) == 1);

In [308]:
l = length(V)
@constraint(model, [z in Z], sum(x[j,l,z] for j in V) == 1);

#### (8) (9) (10)

In [309]:
@constraint(model,[c in C, z in Z], tau[c,z]<=T)
@constraint(model,[c in C, z in Z], y[z,c]<=Q);

#### Objective

In [310]:
@objective(model, Min, sum(t[i,j] * x[i,j,z] for  z in Z, i in V, j in V if j != i)); 

In [ ]:
# @objective(model, Min, sum( t[i,j] * x[i,j] for i in V, j in filter(x->x!=i,V)));    # i != j 

In [311]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1972 rows, 13175 columns and 74775 nonzeros
Model fingerprint: 0xbfdd2d71
Variable types: 170 continuous, 13005 integer (13005 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [2e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+05]
Presolve removed 181 rows and 1105 columns
Presolve time: 0.74s
Presolved: 1791 rows, 12070 columns, 51516 nonzeros
Variable types: 85 continuous, 11985 integer (11981 binary)

Root relaxation: objective 1.761615e+02, 432 iterations, 0.06 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  176.16147    0   34          -  176.16147      -     -    1s
     0     0  176.59668    0   38          -  176.59668     

In [200]:
# value.(x)

In [201]:
value.(tau)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6]
    Dimension 2, [1, 2, 3]
And data, a 6×3 Matrix{Float64}:
   0.0   0.0   0.0
   0.0   0.0   0.0
 212.0  -0.0  -0.0
 140.0  -0.0  -0.0
 332.0   0.0   0.0
   0.0   0.0   0.0

In [133]:
solution_summary(model, verbose=true)

* Solver : Gurobi

* Status
  Termination status : INFEASIBLE_OR_UNBOUNDED
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Result count       : 0
  Has duals          : false
  Message from the solver:
  "Model was proven to be either infeasible or unbounded. To obtain a more definitive conclusion, set the DualReductions parameter to 0 and reoptimize."

* Candidate solution
  Objective bound      : 1.00000e+100

* Work counters
  Solve time (sec)   : 5.46537e-01
  Barrier iterations : 0
  Node count         : 3705
